In [59]:
import cv2
import numpy as np
from cv2 import VideoWriter, VideoWriter_fourcc

In [60]:
background_color = 120
R = background_color
G = background_color
B = background_color
A = background_color
background = [R, G, B, A]
frame = cv2.imread('white_background.jpg', cv2.IMREAD_UNCHANGED)
frame = cv2.cvtColor(frame, cv2.COLOR_RGB2RGBA)
car = cv2.imread('car.png', cv2.IMREAD_UNCHANGED)
car_mask = car[:,:,3] == 0
car[car_mask] = background
bus = cv2.imread('bus.png', cv2.IMREAD_UNCHANGED)
bus_mask = bus[:,:,3] == 0
bus[bus_mask] = background
car2 = car
print(car.shape)
print(bus.shape)
x_offset_car = 460
y_offset_car = 400
x_offset_bus = 750
y_offset_bus = 20
x_offset_car2 = 150
y_offset_car2 = 20
frame[y_offset_car:y_offset_car+car.shape[0], x_offset_car:x_offset_car+car.shape[1]] = car
frame[y_offset_bus:y_offset_bus+bus.shape[0], x_offset_bus:x_offset_bus+bus.shape[1]] = bus
frame[y_offset_car2:y_offset_car2+car2.shape[0], x_offset_car2:x_offset_car2+car2.shape[1]] = car2
cv2.imwrite("Generated_frame.png", frame)

(171, 88, 4)
(293, 143, 4)


True

In [61]:
# Initialize video parameters
width = 1024
height = 600
FPS = 30
seconds = 20
initial_y_offset = 400
in_frame = True
boundary = 10
bus_remain = bus.shape[0]
car2_remain = car2.shape[0]
lane_thickness = 10
left_lane = 350
right_lane = 660
# Relative speed data.
bus_speed = 2
car_speed = 1

In [62]:
fourcc = VideoWriter_fourcc(*'MP42')
video = VideoWriter('./HUD.mp4', fourcc, float(FPS), (width, height))
for _ in range(FPS * seconds):
    frame.fill(background_color) # generate new white background
    cv2.line(frame, (left_lane, 0), (left_lane, height), (255,255,255), lane_thickness)
    cv2.line(frame, (right_lane, 0), (right_lane, height), (255,255,255), lane_thickness)
    cv2.line(frame, (40, 0), (40, height), (50,50,50), 20)
    cv2.line(frame, (980, 0), (980, height), (50,50,50), 20)
    car = car
    bus = bus
    car2 = car2
    bus_tail = height - (bus.shape[0] + boundary)
    car2_tail = height - (car2.shape[0] + boundary)
    y_offset_car2 += car_speed
    y_offset_bus += bus_speed
    if (y_offset_bus >= bus_tail and bus_remain > 0):
        bus_remain = (height - y_offset_bus) - boundary
    if (y_offset_car2 >= car2_tail and car2_remain > 0):
        car2_remain = (height - y_offset_car2) - boundary
    frame[y_offset_bus:y_offset_bus+bus_remain, x_offset_bus:x_offset_bus+bus.shape[1]] = bus[0:bus_remain,:]
    frame[y_offset_car2:y_offset_car2+car2_remain, x_offset_car2:x_offset_car2+car2.shape[1]] = car2[0:car2_remain,:]
    frame[y_offset_car:y_offset_car+car.shape[0], x_offset_car:x_offset_car+car.shape[1]] = car
    video.write(frame)
video.release()

In [63]:
'''
Utility class for file conversion.
'''
import imageio
import os, sys
imageio.plugins.ffmpeg.download()

class TargetFormat(object):
    GIF = ".gif"
    MP4 = ".mp4"
    AVI = ".avi"
    
def convert(input_path, target_format):
    output_path = os.path.splitext(input_path)[0] + target_format
    print("Converting\r\n\t{0}\r\nto\r\n\t{1}".format(input_path, output_path))
    reader = imageio.get_reader(input_path)
    fps = reader.get_meta_data()['fps']
    
    writer = imageio.get_writer(output_path, fps=fps)
    for i, frame in enumerate(reader):
        if (i < 80):
            sys.stdout.write("\rframe {0}".format(i))
            sys.stdout.flush()
            writer.append_data(frame)
    print("\r\nFinalizing...")
    writer.close()
    print("Done.")

In [64]:
# Generate GIF for README.md demo.
convert("./HUD.mp4", TargetFormat.GIF)

Converting
	./HUD.mp4
to
	./HUD.gif
frame 79
Finalizing...
Done.
